# Importacion de funciones del script "funciones.ipynb"

In [1]:
import import_ipynb
import pandas as pd
import os
from funciones import get_data, build_table, save_to_csv, date_fromTo, agg_code

importing Jupyter notebook from funciones.ipynb


# Conexion a API.

 Se realiza la conexion a la API, realizando solicitudes GET para obtener datos en formato JSON, que posteriormente serán convertidos en DataFrames.

### Endpoint factors

In [2]:
base_url= 'http://api.carbonintensity.org.uk'
endpoint = 'intensity/factors'
factors = get_data(base_url, endpoint)

if factors:
  df_factors = build_table(factors)

# Reestructuracion de DataFrame
df_factors = df_factors.melt()

# Renombrar columnas
df_factors = df_factors.rename(columns = {'variable':'Fuels'})

In [14]:
df_factors

,Fuels,value
0,Biomass,120
1,Coal,937
2,Dutch Imports,474
3,French Imports,53
4,Gas (Combined Cycle),394
5,Gas (Open Cycle),651
6,Hydro,0
7,Irish Imports,458
8,Nuclear,0
9,Oil,935


### Endpoint intensity

In [4]:
base_url= 'http://api.carbonintensity.org.uk'
endpoint= 'intensity'
params= date_fromTo()

intensity = get_data(base_url, endpoint, params)
if intensity:
  df_intensity = build_table(intensity)

In [5]:
df_intensity

,from,to,intensity.forecast,intensity.actual,intensity.index
0,2023-10-12T04:30Z,2023-10-12T05:00Z,216,273,high
1,2023-10-12T05:00Z,2023-10-12T05:30Z,243,281,high
2,2023-10-12T05:30Z,2023-10-12T06:00Z,265,275,high
3,2023-10-12T06:00Z,2023-10-12T06:30Z,273,278,high
4,2023-10-12T06:30Z,2023-10-12T07:00Z,280,271,high
...,...,...,...,...,...
1345,2023-11-11T02:30Z,2023-11-11T03:00Z,93,103,low
1346,2023-11-11T03:00Z,2023-11-11T03:30Z,92,101,low
1347,2023-11-11T03:30Z,2023-11-11T04:00Z,97,97,low
1348,2023-11-11T04:00Z,2023-11-11T04:30Z,99,97,low


## Endpoint regional

 A partir de este endpoint se busca obtener 4 dataframes distintos, por lo que es necesario realizar transformaciones.


*   **df_date**: Almacena fechas.
*   **df_regions**: Almacena regiones.
*   **df_regional**: Su funcion es almacenar los registros obtenidos mediante el uso de indices.
*   **df_generation**: Almacena percentiles de emision por cada una de las fuentes de energia estudiadas, respectivamente por region y por fecha.


In [6]:
base_url= 'http://api.carbonintensity.org.uk'
endpoint = 'regional/intensity'
params= date_fromTo(8) # 7 dias.

regional = get_data(base_url,endpoint, params)
if regional:
  df_regional = build_table(regional)

In [7]:
# Cambiar de tipo de datos.
df_regional['from'] = pd.to_datetime(df_regional['from'])

# Eliminar columna to
df_regional = df_regional.drop(columns= ['to'])

# Crear columnas de fechas y hora.

df_regional['hour'] = df_regional['from'].dt.time
df_regional['month'] = df_regional['from'].dt.month
df_regional['day'] = df_regional['from'].dt.day
df_regional['year'] = df_regional['from'].dt.year

# Filtrar los registros obtenidos por hora
df_regional = df_regional[df_regional['hour'].apply(lambda x: x.minute == 0)]

# Renombrar columna from a date
df_regional = df_regional.rename(columns = {'from':'date'})

# Resetear indices
df_regional.reset_index(drop=True, inplace= True)


# Se crea una columna que contiene un codigo de identificacion correspondiente a una fecha y una hora determinada.
# El formato corresponde a: COD/año+mes+dia+hora

df_regional['cod_date'] = df_regional['date'].apply(lambda x: 'COD' + x.strftime('%Y%m%d%H'))


### df_date

In [8]:
df_date = df_regional[['date', 'year', 'month', 'day', 'hour', 'cod_date']]
df_date

,date,year,month,day,hour,cod_date
0,2023-11-04 05:00:00+00:00,2023,11,4,05:00:00,COD2023110405
1,2023-11-04 06:00:00+00:00,2023,11,4,06:00:00,COD2023110406
2,2023-11-04 07:00:00+00:00,2023,11,4,07:00:00,COD2023110407
3,2023-11-04 08:00:00+00:00,2023,11,4,08:00:00,COD2023110408
4,2023-11-04 09:00:00+00:00,2023,11,4,09:00:00,COD2023110409
...,...,...,...,...,...,...
163,2023-11-11 00:00:00+00:00,2023,11,11,00:00:00,COD2023111100
164,2023-11-11 01:00:00+00:00,2023,11,11,01:00:00,COD2023111101
165,2023-11-11 02:00:00+00:00,2023,11,11,02:00:00,COD2023111102
166,2023-11-11 03:00:00+00:00,2023,11,11,03:00:00,COD2023111103


### df_regions

In [9]:
# Para la creacion de este DataFrame se hace uso de un único registro debido a que el dato de las regiones son estáticas, o sea nunca cambian.

regions = df_regional['regions'][0]
dict_regions = {
    'region_id' : [],
    'shortname' : []
}

for region in regions:
  id = region['regionid']
  name = region['shortname']
  if id not in dict_regions:
    dict_regions['region_id'].append(id)
    dict_regions['shortname'].append(name)

df_regions = pd.DataFrame(dict_regions)
df_regions

,region_id,shortname
0,1,North Scotland
1,2,South Scotland
2,3,North West England
3,4,North East England
4,5,Yorkshire
5,6,North Wales & Merseyside
6,7,South Wales
7,8,West Midlands
8,9,East Midlands
9,10,East England


### df_regional

In [10]:
serie_reg = df_regional['regions']  # Trabajaremos con la columna 'regions' de df_regional original.
n=0
for raw in serie_reg:
  for i in range(len(raw)):
    code = agg_code(raw[i],df_regional['cod_date'][n])
    raw[i]['cod_date'] = code   # Se crea un código que identifique cada registro segun hora y fecha.
    new_key = {
        'cod_date': code,
        'regionid': raw[i]['regionid']
    }

    raw[i]['generationmix'].append(new_key)

  n = n+1

df_regional = pd.json_normalize(serie_reg)    # Nuevo DataFrame regional.

"""
 El nuevo dataFrame df_regional contiene registros en formato json.
 Se busca crear una lista de DataFrames a partir de esos archivos .json que posteriormente serán concatenados.
"""

list_DF = [pd.json_normalize(df_regional[i]) for i in range(18)] #18 correspondiente a las 18 regiones registradas.
df_regional = pd.DataFrame()

for i in range( len(list_DF) ):
  df_regional = pd.concat([df_regional, list_DF[i]], ignore_index = True)
  
df_regional.rename(columns={'regionid':'region_id'}, inplace=True)
df_regional

,region_id,dnoregion,shortname,generationmix,cod_date,intensity.forecast,intensity.index
0,1,Scottish Hydro Electric Power Distribution,North Scotland,"[{'fuel': 'biomass', 'perc': 5.2}, {'fuel': 'c...",COD2023110405,18,very low
1,1,Scottish Hydro Electric Power Distribution,North Scotland,"[{'fuel': 'biomass', 'perc': 0.9}, {'fuel': 'c...",COD2023110406,11,very low
2,1,Scottish Hydro Electric Power Distribution,North Scotland,"[{'fuel': 'biomass', 'perc': 0.7}, {'fuel': 'c...",COD2023110407,10,very low
3,1,Scottish Hydro Electric Power Distribution,North Scotland,"[{'fuel': 'biomass', 'perc': 0.6}, {'fuel': 'c...",COD2023110408,12,very low
4,1,Scottish Hydro Electric Power Distribution,North Scotland,"[{'fuel': 'biomass', 'perc': 0.8}, {'fuel': 'c...",COD2023110409,13,very low
...,...,...,...,...,...,...,...
3019,18,GB,GB,"[{'fuel': 'biomass', 'perc': 10.7}, {'fuel': '...",COD2023111100,99,low
3020,18,GB,GB,"[{'fuel': 'biomass', 'perc': 9.6}, {'fuel': 'c...",COD2023111101,89,low
3021,18,GB,GB,"[{'fuel': 'biomass', 'perc': 8.5}, {'fuel': 'c...",COD2023111102,91,low
3022,18,GB,GB,"[{'fuel': 'biomass', 'perc': 8.5}, {'fuel': 'c...",COD2023111103,93,low


### df_generation

In [11]:
df_aux = df_regional['generationmix'] # dataframe auxiliar, contiene los percentiles

df_generation = pd.DataFrame()

for raw in df_aux:  # recorre cada registro del dataframe auxiliar.
  df_raw = pd.json_normalize(raw)
  df_raw = df_raw.fillna(df_raw.iloc[-1])[:-1]    #Rellena los valores nulos de las columnas 'cod_date', y 'regionid'.
  df_generation = pd.concat([df_generation, df_raw], ignore_index = True)   #Concatena a medida que itera.
  
df_generation

,fuel,perc,cod_date,regionid
0,biomass,5.2,COD2023110405,1.0
1,coal,0.1,COD2023110405,1.0
2,imports,7.2,COD2023110405,1.0
3,gas,2.6,COD2023110405,1.0
4,nuclear,29.7,COD2023110405,1.0
...,...,...,...,...
27211,nuclear,16.9,COD2023111104,18.0
27212,other,0.0,COD2023111104,18.0
27213,hydro,1.0,COD2023111104,18.0
27214,solar,0.3,COD2023111104,18.0


# Almacenamiento

 Se realiza el almacenamiento de los datos obtenidos, en formato .csv en la carpeta archivos.

In [12]:
# Variable que almacena la ruta de la carpeta que almacenará los datos extraidos, en formato.csv

dir_path = f"{os.path.dirname(os.getcwd())}/archivos"

In [13]:
df_list = [df_factors, df_intensity, df_regional, df_date, df_regions, df_generation]
file_name_list = ['factors.csv', 'intensity.csv', 'regional.csv', 'date.csv', 'regions.csv', 'generation.csv']


for df, file_name in zip(df_list, file_name_list):
    save_to_csv(df, f'{dir_path}/{file_name}')

DataFrame guardado exitosamente en formato CSV.
DataFrame guardado exitosamente en formato CSV.
DataFrame guardado exitosamente en formato CSV.
DataFrame guardado exitosamente en formato CSV.
DataFrame guardado exitosamente en formato CSV.
DataFrame guardado exitosamente en formato CSV.
